In [2]:
import pandas as pd

In [3]:
df_teewts = pd.read_csv("https://github.com/murpi/wilddata/raw/master/quests/tweets.zip")

In [4]:
df_teewts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27480 entries, 0 to 27479
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27480 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27480 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [5]:
df_teewts.head(5)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


Entraîne-toi à classifier des tweets pour dire s'ils sont positifs ou négatifs. Ca pourrait être un outil très utile pour optimiser le travail d'un community manager.

1/ Importe cet ensemble de données de tweets dans un DataFrame.


In [6]:
df_teewts['sentiment'].value_counts(normalize=True).mul(100).round(2)

sentiment
neutral     40.45
positive    31.23
negative    28.32
Name: proportion, dtype: float64


2/ Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral). Quel est le pourcentage de tweets positifs/négatifs ?

In [7]:
df_0 = df_teewts[~df_teewts['sentiment'].str.contains("neutral")]
df_0['sentiment'].value_counts(normalize=True).mul(100).round(2)

sentiment
positive    52.45
negative    47.55
Name: proportion, dtype: float64

3/ Copie la colonne text dans une Série X, et la colonne sentiment dans une Série y. Applique un train test split avec le random_state = 32 et un train_size de 0.75.

4/ Crée un modèle vectorizer avec scikit-learn en utilisant la méthode TfidfVectorizer. Entraîne ton modèle sur X_train, puis crée une matrice de features X_train_CV. Crée la matrice X_test_CV sans ré-entraîner le modèle. Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements.

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

tfidf = TfidfVectorizer()

X = df_0['text']
y = df_0['sentiment']

X_train,X_test, y_train, y_test = train_test_split(X, y, random_state=32, train_size=0.75)


tfidf.fit(X_train, y_train)

TfidfVectorizer()

In [11]:
X_train_CV = tfidf.transform(X_train)
X_train_CV

<12272x15806 sparse matrix of type '<class 'numpy.float64'>'
	with 144578 stored elements in Compressed Sparse Row format>

In [12]:
X_test_CV = tfidf.transform(X_test)
X_test_CV

<4091x15806 sparse matrix of type '<class 'numpy.float64'>'
	with 44633 stored elements in Compressed Sparse Row format>

5/ Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : 0.932pour le test d'entraînement, et 0.873 pour l'ensemble de test.

In [13]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(X_train_CV,y_train)

model.score(X_train_CV, y_train), model.score(X_test_CV, y_test)

(0.9313070404172099, 0.8731361525299438)